In [ ]:
!pip install pdf2image
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from google.colab import files
from PIL import Image
from torch.utils.data import Dataset
from pdf2image import convert_from_path
import torchvision.transforms as transforms

class FilenameLabelDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.samples = []

        for file in os.listdir(folder_path):
            if file.endswith((".jpg", ".png", ".jpeg")):
                label = 0 if "Accepted" in file else 1
                self.samples.append((os.path.join(folder_path, file), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_path = "/content/drive/MyDrive/SAFE_AI(Project_proposal)/ICLR_Dataset_LowRes/train"
test_path = "/content/drive/MyDrive/SAFE_AI(Project_proposal)/ICLR_Dataset_LowRes/test"

train_dataset = FilenameLabelDataset(train_path, transform=transform)
test_dataset = FilenameLabelDataset(test_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
class VGG16Model(nn.Module):    # applying VGG16 model
    def __init__(self):
        super(VGG16Model, self).__init__()
        base_model = models.vgg16(pretrained=True)

        self.conv = base_model.features
        self.pool = nn.AdaptiveAvgPool2d((7, 7))  # set max pooling output by 7x7

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 2)  # good or bad paper -> binary classification
        )

        for param in self.conv.parameters():
            param.requires_grad = False

    def forward(self, x):
        x = self.conv(x)
        x = self.pool(x)
        x = self.fc(x)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG16Model().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # low IR is recommended for VGG

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    print(f"[Epoch {epoch+1}] Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_acc:.2f}%")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[Epoch 1] Loss: 0.5868, Accuracy: 76.13%
[Epoch 2] Loss: 0.1329, Accuracy: 95.83%
[Epoch 3] Loss: 0.0533, Accuracy: 99.17%
[Epoch 4] Loss: 0.0330, Accuracy: 99.50%
[Epoch 5] Loss: 0.0282, Accuracy: 99.67%


In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_acc = 100 * correct / total
print(f"테스트 정확도: {test_acc:.2f}%")

테스트 정확도: 83.59%


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/SAFE_AI(Project_proposal)/Models/ICLR_vgg_model.pt")